In [8]:
%%time
import csv
import random
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression

#this part is just the file that later on assigns words to CUIs
f555 = open('/Users/christopherpan 1/Desktop/Topic Modeling Project Part 3/LEGIT_test_db_dict.txt','r')

dictList = []
for line in f555:
    ll = line.split('|')
    dictList.append([ll[0],ll[1]])
f555.close()


cui_vecs = '/Users/christopherpan 1/Desktop/Topic Modeling Project Part 4/topic model ra legit/raTags.csv'
#tags (cosine similarity) relative to CUI
rows = []
with open(cui_vecs, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    first = True
    for row in reader:
        if first:
            first = False
            continue
        else:
            rows.append(row)
            
rows.sort(key=lambda x: x[2], reverse = True)

#professor wants 50 samples -> get count of each variable generated (kind of like when the CUIs were counted)
for iiiiii in range(0,50):

    sample = []
    indexx = 0

    for row in rows:
        r = random.random()
        if r <= float(row[2]) and float(row[2]) > 0.4:
            sample.append([row[0],row[2]])
            indexx += 1
        #special case for 0.1-0.4 range: divide cos_sim by 4
        elif float(row[2]) <= 0.4 and float(row[2]) >= 0.1 and r < float(row[2])/4:
            sample.append([row[0],row[2]])
            indexx += 1
        #special case for <0.1 range: divide cos_sim by 2
        elif r <= float(row[2])/2.0 and float(row[2]) < 0.1:
            sample.append([row[0],row[2]])
            indexx += 1
    sample.insert(0,['Filename','Tag'])

    path = '/Users/christopherpan 1/Desktop/Topic Modeling Project Part 5/ra/' + str(iiiiii) + '/'

    if not os.path.exists(path):
        os.makedirs(path)
    
    f = open(os.path.join(path,'raSample.csv'), 'w')
    with f:
        writer = csv.writer(f)
        writer.writerows(sample)
    f.close()
    
    #these prints were just for me to see where the program is
    #checkpoint0
    print('sample ' + str(iiiiii))
    
    df = pd.read_csv(path + 'raSample.csv',sep=",")
    filenames = df['Filename']
    tag = df['Tag']
    tagsList = []
    for i in range(indexx):
        tagsList.append([filenames[i],tag[i]])
    filenamesList = []
    for fi in filenames:
        filenamesList.append(fi.replace('.txt',''))

    f = open(os.path.join(path,'LEGIT_RESULTS.txt'), 'w')
    
    #checkpoint1
    print('results ' + str(iiiiii))
    
    #the sample generated only has filenames, this part gets the CUI counts of only these files (cont.)
    #from the total data file (NLP output data e.g. test_db_res.txt)
    
    with open('/Users/christopherpan 1/Desktop/Topic Modeling Project Part 3/test_db_res.txt','r') as file1:
        data = file1.readlines()
        for line in data:
            title = line.split('|')[0]
            if title in filenamesList:
                a = line.replace(':YES','')
                b = a.replace(':NO','')
                c = b.replace(':UNCLEAR','')
                f.write(c)

    f.close()
    theTitles = []
    #only get CUIs with >0.1 cos_sim to disease
    with open('/Users/christopherpan 1/Desktop/Topic Modeling Project Part 4/topic model ra/ra0.1.txt','r') as file3:
        data1 = file3.readlines()
        for line in data1:
            theTitles.append(line.split(':')[0])
            cuislol = []
    with open('/Users/christopherpan 1/Desktop/Topic Modeling Project Part 4/topic model ra/ra0.1.txt','r') as file3:
        data1 = file3.readlines()
        for line in data1:
            cuislol.append([line.split(':')[0],line.split(':')[1].replace('\n','')])
    theDictlist = []
    with open(path + 'LEGIT_RESULTS.txt','r') as file2:
        data = file2.readlines()
        for line in data:
            l = line.split('|')
            cuis = l[2].split(',')
            for cui in cuis:
                cui = cui.replace('\n','')
                if cui in theDictlist:
                    continue
                elif cui in theTitles:
                    theDictlist.append(cui)
    theDictlist.sort()
    theDictlistCUIs = []
    for d in theDictlist:
        for cu in cuislol:
            if cu[0] == d:
                theDictlistCUIs.append([cu[0],cu[1]])
    res = open(path + 'LEGIT_RESULTS.txt','r')
    f2 = open(os.path.join(path,'CUICountsra.txt'), 'w')
    results = res.readlines()
    f2.write('filename')
    for d in theDictlistCUIs:
        f2.write(',' + d[0]+'|'+ d[1])
    f2.write('\n')
    for line in results:
        l = line.split('|')
        title = l[0].replace(',','')
        CUIs = l[2].split(',')
        f2.write(title)
        for di in theDictlist:
            count = 0
            for cui in CUIs:
                if cui.replace('\n','') == di:
                    count += 1
            f2.write(',' + str(count))
        f2.write('\n')
    f2.close()
    
    #checkpoint2
    print('counts ' + str(iiiiii))
    #table with counts of all CUIs generated
    
    df = pd.read_csv(path + 'CUICountsra.txt',sep=",")
    filenamesList = []
    filenames = df['filename']
    for fil in filenames:
        filenamesList.append(fil)
    df2 = pd.read_csv(path + 'raSample.csv',sep=",")
    tag = df2['Tag']
    filnames = df2['Filename']
    tagsList = []
    i = 0
    for i in range(indexx):
        tagsList.append([filnames[i],tag[i]])
    tagsList.sort(key=lambda x: x[0])
    tagAppend = []
    for tag in tagsList:
        tagAppend.append(tag[1])
    df.insert(1, 'Cos_Sim', pd.Series(tagAppend))
    df.sort_values('filename')
    df = df.sort_values('Cos_Sim',ascending=False)
    df.to_csv(path + 'raData.csv', sep=',',header=True)

    #checkpoint3
    print('data ' + str(iiiiii))
    #tags appended
    
#     # tag > 0.5 = 1, others are 0
#     df['Bin_tag'] = (df['Cos_Sim'] > 0.5)
#     df['Bin_tag'] = df['Bin_tag'].astype(object).replace({False:'0',True:'1'})
#     clean_frame_train = df.drop(['filename','Cos_Sim'],axis = 1,inplace = False)
#     clean_frame_train = clean_frame_train.drop(clean_frame_train.columns[0],axis = 1,inplace = False)
#     y = clean_frame_train.Bin_tag
#     X_train, X_valid, y_train, y_valid = train_test_split(clean_frame_train, y, test_size=0.20)
    
#     X_tr = X_train.drop(['Bin_tag'], axis=1)
#     X_vl = X_valid.drop(['Bin_tag'], axis=1)
    
#     #normalize data (?)
#     #X_tr = normalize(X_tr.as_matrix())
#     #X_vl = normalize(X_vl.as_matrix())

#     y_tr = y_train.as_matrix()
#     y_vl = y_valid.as_matrix()
#     regr = LogisticRegression()
    
#     #try C values in this list, pick one that gets best score with validation data (I think 10 is almost always picked, haven't looked at all yet)
#     ccs = [0.001,0.01,0.1,1,10]
#     theC = 0.001
#     runningScore = -1
#     for c in ccs:
#         regr.set_params(penalty ='l1',C = c)
#         regr.fit(X_tr,y_tr)
#         a = regr.predict(X_vl)
#         score = regr.score(X_vl, y_vl)
#         if score > runningScore:
#             runningScore = 0
#             theC = c
#     regr = LogisticRegression()
#     regr.set_params(penalty = 'l1', C = theC)
#     regr.fit(X_tr,y_tr)

#     #checkpoint4
#     print('regression ' + str(iiiiii))
    
#     #get coefficients
#     coefficients = regr.coef_[0]
#     clean_frame_train_2 = clean_frame_train.drop(['Bin_tag'], axis = 1,inplace=False)
#     pd.Series(coefficients, index=clean_frame_train_2.columns)
#     coefficientList = pd.Series(coefficients, index=clean_frame_train_2.columns)
#     llll = len(coefficientList)
#     coeffList = []
#     for i in range(llll):
#         if coefficientList[i] != 0.0:
#             coeffList.append([clean_frame_train_2.columns[i],coefficientList[i]])
#     coeffList.sort(key=lambda x: abs(float(x[1])), reverse = True)
#     f3 = open(path + str(iiiiii) + 'tiaWords.txt','w')
#     f3.write('Disease:Transient Ischemic Attack\n')
#     f3.write('C = ' + str(theC) +'\n')
#     for l in coeffList:
#         ll = l[0].split('|')[0]
#         for lin in dictList:
#             if lin[1] == ll:
#                 f3.write(l[0] + ',' + str(l[1]) + ' ' + lin[0] + '\n')
#                 break
#     f3.close()

#     #checkpoint5
    print('ALL ' + str(iiiiii))


sample 0
results 0
counts 0
data 0
ALL 0
sample 1
results 1
counts 1
data 1
ALL 1
sample 2
results 2
counts 2
data 2
ALL 2
sample 3
results 3
counts 3
data 3
ALL 3
sample 4
results 4
counts 4
data 4
ALL 4
sample 5
results 5
counts 5
data 5
ALL 5
sample 6
results 6
counts 6
data 6
ALL 6
sample 7
results 7
counts 7
data 7
ALL 7
sample 8
results 8
counts 8
data 8
ALL 8
sample 9
results 9
counts 9
data 9
ALL 9
sample 10
results 10
counts 10
data 10
ALL 10
sample 11
results 11
counts 11
data 11
ALL 11
sample 12
results 12
counts 12
data 12
ALL 12
sample 13
results 13
counts 13
data 13
ALL 13
sample 14
results 14
counts 14
data 14
ALL 14
sample 15
results 15
counts 15
data 15
ALL 15
sample 16
results 16
counts 16
data 16
ALL 16
sample 17
results 17
counts 17
data 17
ALL 17
sample 18
results 18
counts 18
data 18
ALL 18
sample 19
results 19
counts 19
data 19
ALL 19
sample 20
results 20
counts 20
data 20
ALL 20
sample 21
results 21
counts 21
data 21
ALL 21
sample 22
results 22
counts 22
data 22

In [9]:
## %%time
import csv
import math
import random
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model

f555 = open('/Users/christopherpan 1/Desktop/Topic Modeling Project Part 3/LEGIT_test_db_dict.txt','r')

dictList = []
for line in f555:
    ll = line.split('|')
    dictList.append([ll[0],ll[1]])
f555.close()

for iiiiii in range(0,50):
    path = '/Users/christopherpan 1/Desktop/Topic Modeling Project Part 5/ra/' + str(iiiiii) + '/'
    path2 = '/Users/christopherpan 1/Desktop/Topic Modeling Project_SGD/ra/' + str(iiiiii) + '/'

    if not os.path.exists(path2):
        os.makedirs(path2)
    
    df = pd.read_csv(path + 'raData.csv',sep=",")
    df = df.drop(df.columns[0], axis=1, inplace = False)
    #checkpoint3
    print('data ' + str(iiiiii))
    #tags appended
    
    # tag > 0.5 = 1, others are 0
    df['Bin_tag'] = (df['Cos_Sim'] > 0.5)
    df['Bin_tag'] = df['Bin_tag'].astype(object).replace({False:'0',True:'1'})
    clean_frame_train = df.drop(['filename','Cos_Sim'],axis = 1,inplace = False)
    clean_frame_train = clean_frame_train.drop(clean_frame_train.columns[0],axis = 1,inplace = False)
    y = clean_frame_train.Bin_tag
    X_train, X_valid, y_train, y_valid = train_test_split(clean_frame_train, y, test_size=0.20)
    
    X_tr = X_train.drop(['Bin_tag'], axis=1)
    X_vl = X_valid.drop(['Bin_tag'], axis=1)
    
#     numeric_cols = [col for col in X_tr if X_tr[col].dtype.kind != 'O']
#     X_tr[numeric_cols] += 1
#     X_tr = np.log(X_tr)
    
    #normalize data (?)
    X_tr = normalize(X_tr.as_matrix())
    X_vl = normalize(X_vl.as_matrix())

    y_tr = y_train.as_matrix()
    y_vl = y_valid.as_matrix()
    
    #try C values in this list, pick one that gets best score with validation data (I think 10 is almost always picked, haven't looked at all yet)
#     ccs = [0.001,0.01,0.1,1,10]
#     theC = 0.001
#     runningScore = -1
#     for c in ccs:
#         regr.set_params(penalty ='l1',C = c)
#         regr.fit(X_tr,y_tr)
#         a = regr.predict(X_vl)
#         score = regr.score(X_vl, y_vl)
#         if score > runningScore:
#             runningScore = 0
#             theC = c
#     regr = LogisticRegression()
#     regr.set_params(penalty = 'l1', C = theC)
#     regr.fit(X_tr,y_tr)
    
    regr = linear_model.SGDClassifier(loss='log', penalty='elasticnet', alpha=.001, l1_ratio=0.15)
    regr.fit(X_tr,y_tr)

    #checkpoint4
    print('regression ' + str(iiiiii))
    
    #get coefficients
    coefficients = regr.coef_[0]
    clean_frame_train_2 = clean_frame_train.drop(['Bin_tag'], axis = 1,inplace=False)
    pd.Series(coefficients, index=clean_frame_train_2.columns)
    coefficientList = pd.Series(coefficients, index=clean_frame_train_2.columns)
    llll = len(coefficientList)
    coeffList = []
    for i in range(llll):
        if coefficientList[i] != 0.0:
            coeffList.append([clean_frame_train_2.columns[i],coefficientList[i]])
    coeffList.sort(key=lambda x: abs(float(x[1])), reverse = True)
    f3 = open(path2 + str(iiiiii) + 'raWords.txt','w')
    f3.write('Disease:Rheumatoid Arthritis\n')
    f3.write('SGDClassifier: alpha = 0.001, l1_ratio = 0.15\n')
    for l in coeffList:
        ll = l[0].split('|')[0]
        for lin in dictList:
            if lin[1] == ll:
                f3.write(l[0] + ',' + str(l[1]) + ' ' + lin[0] + '\n')
                break
    f3.close()

    #checkpoint5
    print('ALL ' + str(iiiiii))


data 0


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:51: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter wil

regression 0
ALL 0
data 1
regression 1
ALL 1
data 2
regression 2
ALL 2
data 3
regression 3
ALL 3
data 4
regression 4
ALL 4
data 5
regression 5
ALL 5
data 6
regression 6
ALL 6
data 7
regression 7
ALL 7
data 8
regression 8
ALL 8


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


data 9
regression 9
ALL 9
data 10
regression 10
ALL 10
data 11
regression 11
ALL 11
data 12
regression 12
ALL 12
data 13
regression 13
ALL 13
data 14
regression 14
ALL 14
data 15
regression 15
ALL 15
data 16
regression 16
ALL 16
data 17
regression 17
ALL 17
data 18
regression 18
ALL 18
data 19
regression 19
ALL 19
data 20
regression 20
ALL 20
data 21
regression 21
ALL 21
data 22
regression 22
ALL 22
data 23
regression 23
ALL 23
data 24
regression 24
ALL 24
data 25
regression 25
ALL 25
data 26
regression 26
ALL 26
data 27
regression 27
ALL 27
data 28
regression 28
ALL 28
data 29
regression 29
ALL 29
data 30
regression 30
ALL 30
data 31
regression 31
ALL 31
data 32
regression 32
ALL 32
data 33
regression 33
ALL 33
data 34
regression 34
ALL 34
data 35
regression 35
ALL 35
data 36
regression 36
ALL 36
data 37
regression 37
ALL 37
data 38
regression 38
ALL 38
data 39
regression 39
ALL 39
data 40
regression 40
ALL 40
data 41
regression 41
ALL 41
data 42
regression 42
ALL 42
data 43
regressio